In [ ]:
import sys
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline


In [ ]:
# Initialize Spark session
spark = SparkSession.builder.appName("OHLCV Analysis").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)


In [ ]:
# Location of bucket that contains market data parquet files


# Read prices parquet file into a DataFrame (OHLCV market data)
parquet_file_path = "../../raw_data/sharadar.sep.parquet"
df = spark.read.parquet(parquet_file_path)

# Read ticker metadata file into a Dataframe (company information)
parquet_file_path = "../../raw_data/sharadar.tickers.parquet"
df_t = spark.read.parquet(parquet_file_path)
df_t = df_t.withColumnRenamed('ticker', 'ticker2')
for col in [column for column in df_t.columns if column not in ['exchange', ',currency', 'category', 'sector', 'industry', 'ticker2']]:
    df_t = df_t.drop(col)